## Stats701-001 Homework 4: Objects, Classes and Methods
### Taylor Spooner
#### spoonert@umich.edu

**Collaboration**: I did not collaborate with anyone for this assignment.

**Time**: 

### Problem 1: Still More Fun with Vectors
#### 1. Define a class ```Vector```. 
- **Every vector should have a dimension (a non-negative integer) and a list or tuple of its entries.**
- **The initializer for your class should take the dimension as its first argument and a list or tuple of numbers (ints or floats), representing the vector’s entries, as its second argument.** 
- **Choose sensible default behavior for the case where the user applies only a dimension and no entries. **
- **The initializer should raise a sensible error in the case where the dimension is invalid (i.e., wrong type or a negative number), and should also raise an error in the event that the dimension and the number of supplied entries disagree.**

In [3]:
class Vector():
    '''Class represents a vector
    Attributes: a dimension (a non-negative int) and a list of its entries'''
    
    def __init__(self, dim, entries = None):
        '''Initializer for Vector class. Dimension is required and must be a non-negative int.
        The entries are not, default value is a tuple of 0's with length equal to the dimension.'''
        if not isinstance(dim,int):
            raise TypeError("Dimension should be of type int, %s given." % type(dim))
        elif dim < 0:
            raise ValueError("Dimension needs to be non-negative.")
        else:
             self.dim = dim
        
        # Check if entries argument was given or not
        # If not, default it is None, init the vector to all 0's.
        if entries is None:
            self.entries = (0,)*dim
        # If it is given, check if right type
        elif not isinstance(entries, tuple):
            raise TypeError("Entries argument needs to be of type tuple, %s given." % type(entries))
        # Check that all the entries are ints or floats
        elif not all(isinstance(i, (int,float)) for i in entries):
            raise TypeError("All elements of the entries argument must be of type int or float.")
        # Check that the length of entries matches the dim
        elif len(entries) != dim:
            raise ValueError("Number of entries needs to match dimension of vector (dim argument).")
        else:
            self.entries = entries
            
    def dot(self, vec):
        '''Problem 4:
        Arguments:
        - self: a Vector object that the method is getting called on.
        - vec: a Vector object
        Returns: Inner product of two vectors'''
        if not isinstance(vec, Vector):
            raise TypeError("Argument must be of type Vector, %s supplied" % type(vec))
        elif self.dim != vec.dim:
            raise ValueError("Dimensions of two vectors must be equal in length.")
        # Counter for inner sum.
        in_sum = 0
        
        for i in range(self.dim):
            # Compute inner sum
            in_sum = in_sum + self.entries[i]*vec.entries[i]
        return in_sum
    
    def __mul__(self, w):
        '''Problem 5: Multiplication
        Arguments:
        - self: a Vector object that the method is getting called on
        - w: Either a scalar or Vector object to element-wise multiply by self
        Returns: Vector object'''
        # Temp list to hold values at the end
        temp = [1]*self.dim
        if (not isinstance(w, Vector) and not isinstance(w, int) and
            not isinstance(w, float) and not isinstance(w, complex)):
                raise TypeError("Argument w must be of type Vector, float, int or complex.")
        elif isinstance(w, Vector):
            if self.dim != w.dim:
                raise ValueError("The two vectors must have the same dimension.")
            for i in range(self.dim):
                # Multiply the entries together
                temp[i] = self.entries[i]*w.entries[i]
        else:
            # Loop through entries
            for i, val in enumerate(self.entries):
                # Multiply by scalar
                temp[i] = val*w
        return Vector(self.dim, tuple(temp))
    
    def __rmul__(self,w):
        '''Problem 5: Right Multiplication of a Vector with a scalar/Vector'''
        return self.__mul__(w)
    
    def norm(self, p):
        '''Problem 6: 
        Arguemnts: p a non-negative float or int
        Returns the p-norm of the Vector object'''
        if not isinstance(p, int) and not isinstance(p, float):
            raise TypeError("Argument p must be of type float or int.")
        elif p < 0:
            raise ValueError("Argument p must be greater than or equal to 0.")
        elif p == 0:
            # Number of non-zero entries is the total number of elements minus 
            # number of 0's.
            return self.dim - self.entries.count(0)
        # If p is infinity
        elif p == float('Inf'):
            m = -1 # temp max value
            for v in self.entries:
                v_temp = abs(v) # |v_i|^p
                # If this entry is greater than the max, 
                # update max
                if m < v_temp:
                    m = v_tem
            return m
        # p is between 0 and inf
        else:
            rtrn_sum = 0
            for v in self.entries:
                rtrn_sum += abs(v)**p
            return rtrn_sum**(1/p)

In [143]:
v2 = Vector(5)
vars(v2)

{'dim': 5, 'entries': (0, 0, 0, 0, 0)}

In [6]:
v = Vector(0 , ())
vars(v)

{'dim': 0, 'entries': ()}

In [7]:
v3 = Vector(3, (1,2,"3"))

TypeError: All elements of the entries argument must be of type int or float.

In [8]:
v3 = Vector(3, (1,2,3))

#### 2. Did you choose to make the vector’s entries a tuple or a list (there is no wrong answer here, although I would say one is better than the other in this context)? Defend your choice.
I chose to make the vector's entries a tuple. I did this because while I wanted to have the ability to change elements of the entries attribute (mutability in lists), I did not want a user to be able to use the `append` method. If a user appended a new element to the entries attribute (or removed an entry), the entries and dimension would no longer match up. This made me go with tuple. If later in development I find that I really want to change the entries, I guess I could define a function in the `Vector` class to do so.

#### 3. Are the dimension and entries class attributes or instance attributes? Why is this the right design choice?
The dimension and entries attributes are **instance attributes.** When we create a new instance of a Vector() the attributes are owned by that specific instance and not shared by all instances. This is the right design choice because each Vector that we create we will want to have its own dimension and entries. There are no attributes that should have shared, common values across all instances.

#### 4. Implement a method `Vector.dot` that takes a single `Vector` as its argument and returns the inner product of the caller with the given `Vector` object. Your method should raise an appropriate error in the event that the argument is not of the correct type or in the event that the dimensions of the two vectors do not agree.

Method was implemented in the Vector class in problem 1. Its use is shown below.

In [133]:
v.dot("cat")

TypeError: Argument must be of type Vector, <class 'str'> supplied

In [9]:
v1 = Vector(3, (1,2,3))
v2 = Vector(3, (4,5,6))
v1.dot(v2)

32

#### 5. We would also like our `Vector` class to support scalar multiplication. Left- or right multiplication by a scalar, e.g., `2*v` or `v*2`, where `v` is a `Vector` object, should result in a new `Vector` object with its entries all scaled by the given scalar. We will also follow R and numpy (which you will learn in a few weeks), and use `*` to denote entrywise vector-vector multiplication, so that for `Vector` objects `v` and `w`, `v*w` results in a new `Vector` object, with the i-th entry of `v*w` equal to the i-th entry of `v` multiplied by the i-th entry of `w`. Implement the appropriate operators to support this multiplication operation. Many languages have a convention for dealing with multiplication of vectors that differ in their dimension, but we will punt on this matter. Your method should raise an appropriate error in the event that `v` and `w` disagree in their dimensions.

Method was implemented in the Vector class in problem 1. Use is shown below.

In [118]:
v1*"cat"

TypeError: Argument w must be of type Vector, float, int or complex.

In [127]:
v3 = v1*2
print(vars(v3))
v3_2 = 2*v1
print(vars(v3_2))

{'dim': 3, 'entries': (2, 4, 6)}
{'dim': 3, 'entries': (2, 4, 6)}


In [128]:
v4 = v1*v3
print(vars(v4))
v4_2 = v3*v1
print(vars(v4_2))

{'dim': 3, 'entries': (2, 8, 18)}
{'dim': 3, 'entries': (2, 8, 18)}


In [129]:
v_bad = Vector(18)
v5 = v1*v_bad

ValueError: The two vectors must have the same dimension.

#### 6.  Implement a method `Vector.norm` that takes a single int or float `p` as an argument and returns the `p`-norm of the calling `Vector` object. Your method should work whether p is an integer or float. Your method should raise a sensible error in the event that `p` is negative.

For a real number $0 \leq p \leq \infty$, and a vector $v \in \mathbb{R}^d$, the p-norm of $v$, written $\lVert v \rVert_p$, is given by: 

$$\lVert v \rVert_p = \begin{cases} 
      \sum_{i=1}^d 1_{v_i \neq 0} & \text{if}\; p = 0 \\
      \left(\sum_{i=1}^d \left|v_i\right|^p \right)^{1/p} & \text{if}\; 0 < p <\infty \\
      \text{max}_{i=1,\cdots,d} \left|v_i\right| & p = \infty 
   \end{cases}$$
   
Solution was added to the `Vector` class above.

In [140]:
v1.norm(0)

3

In [164]:
v_dec = Vector(3, (.5, .25, .33))
v_dec.norm(float('Inf'))

0.5

In [167]:
v1.norm(4)

3.1463462836457885

### Problem 2: Objects and Classes: Geometry Edition
#### 1. Implement a class `Point` that represents a point in 2-dimensional Euclidean space. Your object should include an initialization method that takes two arguments (with sensibly-chosen default values).

In [24]:
class Point():
    '''Represents a 2-d point'''
    
    def __init__(self, x = 0, y = 0):
        '''Initializer for Point class. x and y must be ints or floats.
        Default point is (0,0).'''
        if not isinstance(x, int) and not isinstance(x, float):
            raise TypeError("Argument x must be of type int or float.")
        elif not isinstance(y, int) and not isinstance(y, float):
            raise TypeError("Argument y must be of type int or float.")
        self.x = x
        self.y = y
    
    def __eq__(self, other):
        '''Equal operator'''
        if not isinstance(other, Point):
            raise TypeError("Cannot compare a Point with a non-Point object.")
        elif self.x == other.x and self.y == other.y:
            return True
        else:
            return False
        
    def __neq__(self, other):
        '''Overrides not equal to.'''
        return not self.__eq__(other)
    
    def __lt__(self, other):
        '''Less than operator'''
        if not isinstance(other, Point):
            raise TypeError("Cannot compare a Point with a non-Point object.")
        if self.x < other.x:
            return True
        elif self.x == other.x and self.y < other.y:
            return True
        else:
            return False
    
    def __le__(self, other):
        '''Less than or equal to'''
        if not isinstance(other, Point):
            raise TypeError("Cannot compare a Point with a non-Point.")
        if self.x < other.x:
            return True
        elif self.x == other.x and self.y <= other.y:
            return True
        else:
            return False
        
    def __gt__(self, other):
        '''Greater than operator'''
        if not isinstance(other, Point):
            raise TypeError("Cannot compare a Point with a non-Point object.")
        if self.x > other.x:
            return True
        elif self.x == other.x and self.y > other.y:
            return True
        else:
            return False

    def __ge__(self, other):
        '''Greater than or equal to'''
        if not isinstance(other, Point):
            raise TypeError("Cannot compare a Point with a non-Point object.")
        if self.x > other.x:
            return True
        elif self.x == other.x and self.y >= other.y:
            return True
        else:
            return False
        
    def __add__(self, other):
        if not isinstance(other, Point):
            raise TypeError("Cannot add a Point object to a non-Point object.")
        x = self.x + other.x
        y = self.y + other.y
        return Point(x,y)

In [25]:
p = Point()
vars(p)

{'x': 0, 'y': 0}

In [26]:
p2 = Point(3,6)
vars(p2)

{'x': 3, 'y': 6}

#### 2. Implement the necessary operator(s) to support comparison (equality, less than, less or equal to, greater than, etc) of Point objects. We will say that two Point objects are equivalent if they have the same x- and y-coordinates. Otherwise, comparison should be analogous to tuples in Python, so that comparison is done on x-coordinates first, and then on y-coordinates. So, for example, the point (2, 4) is ordered before (less than) (2, 5).

In [9]:
p == p2

False

In [10]:
Point() == p

True

In [14]:
p != p2

True

In [18]:
p < p2

True

In [22]:
p <= p2

True

In [23]:
p > p2

False

#### 3. Implement the operator to allow addition of points. That is, if `p1` and `p2` are `Point` objects, then `p1 + p2` should be supported, and should return the `Point` corresponding to adding `p1` and `p2` entrywise (i.e., vector addition).

In [28]:
vars(p + p2)

{'x': 3, 'y': 6}

####  4. Implement a class `Line` that represents a line in the 2-dimensional Euclidean plane. Implement an initialization method that takes a slope and a y-intercept as its two arguments. There are, of course, any number of ways to represent a line, and you are free to choose among them as you like, though of course the slope-intercept representation is most natural given this initializer.

In [29]:
class Line():
    '''Represents a line in the 2-d Euclidean plane.'''
    
    def __init__(self, m = 1, b = 0):
        '''Initializer for the Line object. 
        m is the slope of the line, must be an int or float, and is defaulted as 1.
        b is the y-intercept, must be an int or float, and is defaulted as 0.'''
        if not isinstance(m, int) and not isinstance(m, float):
            raise TypeError("Argument m must be of type int or float.")
        elif not isinstance(b, int) and not isinstance(b, float):
            raise TypeError("Argument b must be of type int or float.")
        self.m = m
        self.b = b
    
    def project(p):
        '''Projection of point onto the line object.'''
        if not isinstance(p, Point):
            raise TypeError("Projection can only be done with type Point.")
        

#### 5. Implement a method `Line.project` that takes a `Point` object as its only argument and returns a `Point` object representing the projection of the argument `Point` object onto the line represented by the caller. Note: this method should create and return a new `Point` object rather than modifying the argument.

Method implemented in the above class. 

### Problem 3: Objects and Inheritance
#### 1. For starters, we can’t have a bibliography without authors. Define a class `Author`, with the following attributes:  
- **`given_name`, a string representing the given name (i.e., first name in English) of the author.**
- **`family_name`, a string representing the family name (i.e., last name in English) of the author.**
- **`author_id`, an integer that will uniquely identify this author**

**Write an initializer for this class that takes a first and last name as its arguments. These should both default to None. Your class should include a class attribute called `next_id`, which is an integer, and is initially zero. Upon initialization of a new Author object, the new object’s `author_id` attribute should be taken to be the current value of `next_id`, and then the class attribute should be incremented so that the next ID we give out is distinct.**

In [5]:
class Author():
    # The id to be given to the author
    next_id = 0
    
    def __init__(self, given_name = None, family_name = None):
        '''Initializer for Author objects.
        - given_name, a string representing the given name (i.e., first name in English) of the author.
        - family_name, a string representing the family name (i.e., last name in English) of the author.
        - author_id, an integer that will uniquely identify this author'''
        if given_name is None:
            self.given_name = ''
        if family_name is None:
            self.family_name = ''
        # If arguments were given, set them equal
        try:
            self.given_name = str(given_name)
            self.family_name = str(family_name)
        except TypeError:
            print("Arguments must be strings or must be able to be converted to strings.")
            raise
        
        self.author_id = self.next_id
        type(self).next_id += 1
        
    def __str__(self):
        return self.family_name + ', ' + self.given_name[0:1] + '.'
        

In [6]:
a = Author("Taylor", "Spooner")
vars(a)

{'author_id': 0, 'family_name': 'Spooner', 'given_name': 'Taylor'}

In [7]:
a2 = Author("Byoungwook", "Wang")
vars(a2)

{'author_id': 1, 'family_name': 'Wang', 'given_name': 'Byoungwook'}

#### 2. Implement the `__str__` operator for the `Author` method. The string representation of an author with first name John and last name Smith should be ’Smith, J.’. That is, the string format should be the family name, followed by first initial. You may assume that all authors have only a first and last name, so that for our purposes, Richard W. Hamming is simply named Richard Hamming.

In [49]:
print(a)

Spooner, T.


In [50]:
print(a2)

Wang, B.


#### 3. The basic unit of a bibliography is a document. For our purposes, we will assume that every document has an author, a title and a year of publication. Define a class `Document`, with the following attributes:
- **`author`, a list of `Author` objects.**
- **`title`, is a string.**
- **`year`, an integer representing a year in the Gregorian calendar.**  

**Implement an initializer for this object, which takes a list of authors, title and year
as its three arguments, in that order. The author list should default to the empty
list, and the other two arguments should default to `None`.**

In [2]:
class Document():
    
    def __init__(self, author=[], title=None, year=None):
        if not isinstance(author, list):
            raise TypeError("Author argument should be of type list.")
        elif not isinstance(year, int) and year is not None:
            raise TypeError("Year argument should be of type int.")
        elif not isinstance(title, str) and title is not None:
            raise TypeError("Title argument should be of type string.")
        # Check that each element in the author object is an Author object.
        for a in author:
            if not isinstance(a, Author):
                raise TypeError("All authors in the list must be of type Author.")
        
        self.author = author
        self.title = title
        self.year = year
    
    def __str__(self):
        '''Problem 4: Print statement for Document class.'''
        # If any of the attributes is None (or empty list)
        # raise error
        if self.author == [] or self.title is None or self.year is None:
            raise ValueError("At least one of the document attributes is None.")
        authors = self.author # Get list of authors
        auth_li = [None]*len(authors)
        # Add the formatted authors to the list.
        for a in range(len(authors)):
            auth_li[a] = authors[a].__str__()
        # Now join the authors together with the word and.    
        auths = " and ".join(auth_li)
        return auths + " (" + str(self.year) + "). " + self.title + "."

In [63]:
d = Document([Author("Taylor", "Spooner"), Author("Byoungwook", "Jang"),Author("Zack", "Keller")], "STATS 701", 2018)
vars(d)

{'author': [<__main__.Author at 0x1c50f7134a8>,
 'title': 'STATS 701',
 'year': 2018}

#### 4.  Implement the `__str__` operator for the `Document` class. The string representation for a document titled Principia by author named Isaac Newton in the year 1687 should be: ’Newton, I. (1687). Principia.’ If there is more than one author in a document, they should be listed in the order that they are listed in the document, separated by the word “and”. So the string representation for a document titled Principia Mathematica by authors Alfred Whitehead and Bertrand Russell in 1910 should be ’Whitehead, A. and Russell, B. (1910). Principia Mathematica.’. Authors should be separated by the word “and” no matter how many authors are in the list `Document.author`. If any of the attributes of a document is not specified (i.e., is `None`), the `__str__` operator should raise a `ValueError`.

In [64]:
d2 = Document()
vars(d2)

{'author': [], 'title': None, 'year': None}

In [65]:
print(d)

Spooner, T. and Jang, B. and Keller, Z. (2018). STATS 701.


#### 5. Implement a class `Book` that inherits from `Document`, but has an additional attribute `publisher`, a string naming a publishing house. 

- **Override the initializer to now take four arguments, the fourth being the string representing the publisher, and the first three being in the same order as for the `Document` initializer.** 
- **Override the `__str__` method to print the same formatted string, but with the publisher name added to the end. So, in the example above, if Principia Mathematica were published by Cambridge University Press, then the string representation would be 'Whitehead, A. and Russell, B. (1910). Principia Mathematica. Cambridge University Press.'**

In [29]:
class Book(Document):
    '''Book class that inherits from the Document class'''
    
    def __init__(self, author=[], title=None, year=None, publisher=None):
        '''Initializer that overrides the Document's init function'''       
        # Use super to use the init function.
        super(Book, self).__init__(author, title, year)
        # Check that publisher is a string
        if not isinstance(publisher, str) and publisher is not None:
            raise TypeError("Publisher argument must be of type string.")
        # Update the publisher
        self.publisher = publisher

    def __str__(self):
        '''String method that uses the inherited str method from document
        but adds the publisher to the end.'''
        if self.publisher is None:
            raise ValueError("Publisher argument is None, cannot print.")
        # Print the same string function as the Document class and add the publisher.
        return(super(Book, self).__str__() + " " + self.publisher + ".")
    

In [27]:
b = Book([a, a2], "STATS 701", 2018, "University of Michigan")
print(b)

Spooner, T. and Wang, B. (2018). STATS 701. University of Michigan.


In [28]:
b = Book([a, a2], "This example won't work", 2018.22, "University of Michigan")

TypeError: Year argument should be of type int.